In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow.keras.utils as ku
import tensorflow as tf
import numpy as np
import re

In [3]:
tokenizer = Tokenizer()

data = open('texts/fellowship-complete.txt').read()

data = data.replace(',', 'pausesentence')
data = data.replace('!', 'exclamationmark')
data = data.replace('?', '.')

corpus = data.lower().split(". ")

for i in range(len(corpus)):
    corpus [i] =  corpus [i] + 'endofthesentence'
    
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:
model = Sequential()

model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(20, return_sequences=True)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(40, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
    
model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 758, 64)           940736    
_________________________________________________________________
bidirectional (Bidirectional (None, 758, 40)           13600     
_________________________________________________________________
dropout (Dropout)            (None, 758, 40)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                9760      
_________________________________________________________________
dense (Dense)                (None, 40)                1640      
_________________________________________________________________
dense_1 (Dense)              (None, 14699)             602659    
Total params: 1,568,395
Trainable params: 1,568,395
Non-trainable params: 0
_________________________________________________________________


In [5]:
split_number = 1723

training_data = []
for i in range(99):
    training_data.append([predictors[i*split_number:(i+1)*split_number], label[i*split_number:(i+1)*split_number]])
training_data.append([predictors[(i+1)*split_number:], label[(i+1)*split_number:]])

In [6]:
model.fit(training_data[10][0], training_data[10][1], epochs=100, verbose=1)

Epoch 1/100
1723/1723 [==============================] - 104s 61ms/step - loss: 8.5978 - acc: 0.0244
Epoch 2/100
1723/1723 [==============================] - 100s 58ms/step - loss: 6.0541 - acc: 0.0441
Epoch 3/100
1723/1723 [==============================] - 100s 58ms/step - loss: 5.8077 - acc: 0.0441
Epoch 4/100
1723/1723 [==============================] - 101s 58ms/step - loss: 5.7513 - acc: 0.0441
Epoch 5/100
1723/1723 [==============================] - 101s 58ms/step - loss: 5.7281 - acc: 0.0435
Epoch 6/100
1723/1723 [==============================] - 100s 58ms/step - loss: 5.7175 - acc: 0.0441
Epoch 7/100
1723/1723 [==============================] - 100s 58ms/step - loss: 5.7024 - acc: 0.0441
Epoch 8/100
1723/1723 [==============================] - 99s 57ms/step - loss: 5.6986 - acc: 0.0441
Epoch 9/100
1723/1723 [==============================] - 100s 58ms/step - loss: 5.6708 - acc: 0.0441
Epoch 10/100
1723/1723 [==============================] - 100s 58ms/step - loss: 5.6310 - ac

1723/1723 [==============================] - 104s 60ms/step - loss: 4.0050 - acc: 0.1091
Epoch 82/100
1723/1723 [==============================] - 100s 58ms/step - loss: 3.9957 - acc: 0.1097
Epoch 83/100
1723/1723 [==============================] - 101s 59ms/step - loss: 3.9510 - acc: 0.1091
Epoch 84/100
1723/1723 [==============================] - 105s 61ms/step - loss: 3.9475 - acc: 0.1161
Epoch 85/100
1723/1723 [==============================] - 103s 60ms/step - loss: 3.9131 - acc: 0.1178
Epoch 86/100
1723/1723 [==============================] - 105s 61ms/step - loss: 3.8922 - acc: 0.1184
Epoch 87/100
1723/1723 [==============================] - 98s 57ms/step - loss: 3.8649 - acc: 0.1201
Epoch 88/100
1723/1723 [==============================] - 93s 54ms/step - loss: 3.8434 - acc: 0.1178
Epoch 89/100
1723/1723 [==============================] - 90s 52ms/step - loss: 3.8340 - acc: 0.1283
Epoch 90/100
1723/1723 [==============================] - 94s 55ms/step - loss: 3.7840 - acc: 0.13

In [7]:
model.save_weights("models/model_complete.h5")

In [8]:
model.fit(training_data[60][0], training_data[60][1], epochs=100, verbose=1)

Epoch 1/100
1723/1723 [==============================] - 86s 50ms/step - loss: 9.6597 - acc: 0.0383
Epoch 2/100
1723/1723 [==============================] - 89s 52ms/step - loss: 7.8399 - acc: 0.0598
Epoch 3/100
1723/1723 [==============================] - 86s 50ms/step - loss: 6.3570 - acc: 0.0575
Epoch 4/100
1723/1723 [==============================] - 87s 51ms/step - loss: 5.8140 - acc: 0.0609
Epoch 5/100
1723/1723 [==============================] - 87s 50ms/step - loss: 5.6148 - acc: 0.0656
Epoch 6/100
1723/1723 [==============================] - 89s 52ms/step - loss: 5.5097 - acc: 0.0650
Epoch 7/100
1723/1723 [==============================] - 86s 50ms/step - loss: 5.4147 - acc: 0.0609
Epoch 8/100
1723/1723 [==============================] - 88s 51ms/step - loss: 5.3583 - acc: 0.0679
Epoch 9/100
1723/1723 [==============================] - 87s 51ms/step - loss: 5.2811 - acc: 0.0714
Epoch 10/100
1723/1723 [==============================] - 88s 51ms/step - loss: 5.2212 - acc: 0.0754

1723/1723 [==============================] - 88s 51ms/step - loss: 2.9385 - acc: 0.2339
Epoch 83/100
1723/1723 [==============================] - 88s 51ms/step - loss: 2.9055 - acc: 0.2356
Epoch 84/100
1723/1723 [==============================] - 90s 52ms/step - loss: 2.8870 - acc: 0.2606
Epoch 85/100
1723/1723 [==============================] - 89s 51ms/step - loss: 2.8606 - acc: 0.2525
Epoch 86/100
1723/1723 [==============================] - 88s 51ms/step - loss: 2.8510 - acc: 0.2542
Epoch 87/100
1723/1723 [==============================] - 87s 50ms/step - loss: 2.8249 - acc: 0.2559
Epoch 88/100
1723/1723 [==============================] - 87s 51ms/step - loss: 2.7919 - acc: 0.2647
Epoch 89/100
1723/1723 [==============================] - 87s 50ms/step - loss: 2.7838 - acc: 0.2583
Epoch 90/100
1723/1723 [==============================] - 86s 50ms/step - loss: 2.7376 - acc: 0.2641
Epoch 91/100
1723/1723 [==============================] - 87s 51ms/step - loss: 2.7965 - acc: 0.2623
Epo

In [9]:
model.save_weights("models/model_complete.h5")

In [11]:
model.fit(training_data[70][0], training_data[70][1], epochs=100, verbose=1)

Epoch 1/100
1723/1723 [==============================] - 84s 49ms/step - loss: 10.5150 - acc: 0.0493
Epoch 2/100
1723/1723 [==============================] - 83s 48ms/step - loss: 9.6498 - acc: 0.0662
Epoch 3/100
1723/1723 [==============================] - 83s 48ms/step - loss: 9.1191 - acc: 0.0760
Epoch 4/100
1723/1723 [==============================] - 83s 48ms/step - loss: 8.5739 - acc: 0.0778
Epoch 5/100
1723/1723 [==============================] - 86s 50ms/step - loss: 7.2737 - acc: 0.0696
Epoch 6/100
1723/1723 [==============================] - 84s 49ms/step - loss: 6.4768 - acc: 0.0633
Epoch 7/100
1723/1723 [==============================] - 84s 49ms/step - loss: 5.9461 - acc: 0.0656
Epoch 8/100
1723/1723 [==============================] - 84s 49ms/step - loss: 5.7391 - acc: 0.0667
Epoch 9/100
1723/1723 [==============================] - 84s 49ms/step - loss: 5.6435 - acc: 0.0702
Epoch 10/100
1723/1723 [==============================] - 84s 49ms/step - loss: 5.5576 - acc: 0.067

1723/1723 [==============================] - 91s 53ms/step - loss: 2.6794 - acc: 0.3012
Epoch 83/100
1723/1723 [==============================] - 117s 68ms/step - loss: 2.6688 - acc: 0.2995
Epoch 84/100
1723/1723 [==============================] - 144s 84ms/step - loss: 2.6190 - acc: 0.3134
Epoch 85/100
1723/1723 [==============================] - 104s 60ms/step - loss: 2.6027 - acc: 0.3076
Epoch 86/100
1723/1723 [==============================] - 88s 51ms/step - loss: 2.6175 - acc: 0.3146
Epoch 87/100
1723/1723 [==============================] - 89s 52ms/step - loss: 2.5495 - acc: 0.3285
Epoch 88/100
1723/1723 [==============================] - 89s 51ms/step - loss: 2.5399 - acc: 0.3175
Epoch 89/100
1723/1723 [==============================] - 126s 73ms/step - loss: 2.4742 - acc: 0.3430
Epoch 90/100
1723/1723 [==============================] - 143s 83ms/step - loss: 2.4359 - acc: 0.3529
Epoch 91/100
1723/1723 [==============================] - 151s 87ms/step - loss: 2.4551 - acc: 0.33

In [13]:
model.save_weights("models/model_complete.h5")

In [ ]:
model.fit(training_data[80][0], training_data[80][1], epochs=100, verbose=1)

In [ ]:
model.save_weights("models/model_complete.h5")

In [ ]:
model.fit(training_data[90][0], training_data[90][1], epochs=100, verbose=1)

In [ ]:
model.save_weights("models/model_complete.h5")

In [ ]:
model.load_weights("models/model_complete.h5")

In [14]:
seed_text = "Speak friend and enter"
next_words = 500
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            if output_word == 'endofthesentence':
                output_word = '.'
            break
    seed_text += " " + output_word
    
seed_text = seed_text.replace('endofthesentence', '.')
seed_text = seed_text.replace('pausesentence', ',')
seed_text = seed_text.replace('exclamationmark', '!')

rtn = re.split('([.!?] *)', seed_text)
seed_text = ''.join([each.capitalize() for each in rtn])

count_sentences = 0
for i in range(len(seed_text)):
    if seed_text[i] == '.':
        count_sentences += 1
        if count_sentences%5 == 0:  
            seed_text = seed_text[:i] + '.\n\n' + seed_text[i+1:].strip()

print(seed_text)

Speak friend and enter you not better. ' then with the drift was rock down with left, and then with the first drift was return to rivendell with rivendell at swimming, and seeing the snow drift ' ached. Southwards, far longer, and the birds was sudden then grows noticed on, grows through langstrand and 'things need boromir. Spades was, we came doomed to go boromir. With answered gandalf.

Behind with answered gandalf. Until we and our satisfied a deep to dare to admit brief and then with gloom to back until you before the mention gandalf. Behind with a rivendell at aragorn with peregrin! Last now,' and evening, and the whiteness. Then with lead and then grows noticed with seeing the power and snow then grows noticed less, food you was then grows flowed at men at men with veiled to above aragorn with gloom veiled to be while less, the head but we and our satisfied the first drift was little men with rivendell at empty and return to rivendell with rivendell at swimming, and forward. Answ